# Antibody Validation with IgFold on Google Colab

Complete pipeline for validating antibody generation models using IgFold.

**Setup:** Enable GPU (Runtime → Change runtime type → GPU)

---

## 1. Check GPU Availability

In [ ]:
import torch

if torch.cuda.is_available():
    print(f"✅ GPU Available: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("❌ No GPU found!")
    print("\n⚠️ IMPORTANT: Enable GPU in Runtime → Change runtime type → GPU (T4)")

## 2. Install Dependencies

**IMPORTANT:** After running this cell, you MUST restart the runtime!
- Click: `Runtime → Restart runtime`
- Then re-run Cell 1
- Skip this cell after restart

In [ ]:
print("Installing IgFold with compatible dependencies...")
print("This may take 3-4 minutes...\n")

# Step 1: Clean uninstall
print("Step 1: Cleaning old packages...")
!pip uninstall -y -q torch torchvision torchaudio transformers accelerate huggingface_hub

# Step 2: Install PyTorch 2.5
print("Step 2: Installing PyTorch 2.5...")
!pip install -q torch==2.5.1 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Step 3: Install compatible huggingface-hub
print("Step 3: Installing huggingface-hub...")
!pip install -q huggingface-hub==0.17.3

# Step 4: Install transformers
print("Step 4: Installing transformers...")
!pip install -q transformers==4.35.0

# Step 5: Install accelerate
print("Step 5: Installing accelerate...")
!pip install -q accelerate==0.20.3

# Step 6: Install IgFold
print("Step 6: Installing IgFold...")
!pip install -q igfold

print("\n✅ Installation complete!")
print("\n⚠️ CRITICAL: You MUST restart the runtime now!")
print("   1. Click: Runtime → Restart runtime")
print("   2. Re-run Cell 1 (GPU check)")
print("   3. Then run Cell 3 (skip Cell 2)")

## 3. Test IgFold with Sample Antibody

In [ ]:
from igfold import IgFoldRunner
import tempfile
import numpy as np

print("Initializing IgFold...")
igfold = IgFoldRunner()
print("✅ IgFold loaded!\n")

# Test with a sample antibody
heavy = "EVQLVESGGGLVQPGGSLRLSCAASGFTISDYAIHWVRQAPGKGLEWVAGITPAGGYTAYADSVKGRFTISADTSKNTAYLQMNSLRAEDTAVYYCARFVFFLPYAMDYWGQGTLVTVSS"
light = "DIQMTQSPSSLSASVGDRVTITCRASQDVSTAVAWYQQKPGKAPKLLIYSASFLYSGVPSRFSGSGSGTDFTLTISSSLQPEDFATYCQQSYTTPPTFGQGTKVEIKR"

print(f"Testing with sample antibody:")
print(f"  Heavy: {len(heavy)} aa")
print(f"  Light: {len(light)} aa")
print("\nPredicting structure... (this may take 30-60 seconds)")

# Create temp file for output
with tempfile.NamedTemporaryFile(mode='w', suffix='.pdb', delete=False) as tmp:
    pdb_file = tmp.name

# Run IgFold
sequences = {'H': heavy, 'L': light}
igfold.fold(pdb_file=pdb_file, sequences=sequences, do_refine=False, do_renum=False)

# Read PDB and extract pLDDT
with open(pdb_file, 'r') as f:
    pdb_string = f.read()

# Extract pLDDT scores from B-factor column
plddt_scores = []
for line in pdb_string.split('\n'):
    if line.startswith('ATOM'):
        try:
            bfactor = float(line[60:66].strip())
            plddt_scores.append(bfactor)
        except:
            pass

plddt_scores = np.array(plddt_scores)

# Convert from fraction (0-1) to percentage (0-100) and clip outliers
plddt_scores = np.clip(plddt_scores * 100, 0, 100)

mean_plddt = plddt_scores.mean()

print(f"\n✅ Structure predicted!")
print(f"   Mean pLDDT: {mean_plddt:.2f}")
print(f"   Min pLDDT:  {plddt_scores.min():.2f}")
print(f"   Max pLDDT:  {plddt_scores.max():.2f}")

if mean_plddt > 70:
    print("\n🎉 Good quality structure!")
elif mean_plddt > 50:
    print("\n✅ Fair quality structure")
else:
    print("\n⚠️ Lower quality structure")

# Save the PDB file
with open('test_antibody.pdb', 'w') as f:
    f.write(pdb_string)
    
print("\nPDB file saved as: test_antibody.pdb")

## 4. Download Test PDB Structure

In [ ]:
from google.colab import files

print("Downloading PDB file...")
files.download('test_antibody.pdb')
print("\n✅ Download complete!")
print("You can view this in PyMOL, ChimeraX, or https://www.rcsb.org/3d-view")

## 5. Validation Function for Multiple Antibodies

In [ ]:
def validate_antibodies(antibody_sequences, save_pdbs=True):
    """
    Validate a list of antibody sequences using IgFold.
    
    Args:
        antibody_sequences: List of tuples (heavy_chain, light_chain)
        save_pdbs: Whether to save PDB files
    
    Returns:
        List of validation results
    """
    from igfold import IgFoldRunner
    import tempfile
    import numpy as np
    import os
    
    print(f"Validating {len(antibody_sequences)} antibodies...\n")
    
    # Initialize IgFold
    igfold = IgFoldRunner()
    
    results = []
    
    for i, (heavy, light) in enumerate(antibody_sequences):
        print(f"[{i+1}/{len(antibody_sequences)}] Processing antibody {i+1}...")
        
        try:
            # Create temp file
            with tempfile.NamedTemporaryFile(mode='w', suffix='.pdb', delete=False) as tmp:
                pdb_file = tmp.name
            
            # Run IgFold
            sequences = {'H': heavy, 'L': light}
            igfold.fold(pdb_file=pdb_file, sequences=sequences, do_refine=False, do_renum=False)
            
            # Read PDB
            with open(pdb_file, 'r') as f:
                pdb_string = f.read()
            
            # Extract pLDDT
            plddt_scores = []
            for line in pdb_string.split('\n'):
                if line.startswith('ATOM'):
                    try:
                        bfactor = float(line[60:66].strip())
                        plddt_scores.append(bfactor)
                    except:
                        pass
            
            plddt_scores = np.array(plddt_scores)
            
            # Convert from fraction (0-1) to percentage (0-100) and clip outliers
            plddt_scores = np.clip(plddt_scores * 100, 0, 100)
            
            mean_plddt = float(plddt_scores.mean())
            
            # Quality grade
            if mean_plddt > 90:
                quality = "Excellent"
            elif mean_plddt > 70:
                quality = "Good"
            elif mean_plddt > 50:
                quality = "Fair"
            else:
                quality = "Poor"
            
            result = {
                'antibody_id': i,
                'heavy_length': len(heavy),
                'light_length': len(light),
                'mean_plddt': mean_plddt,
                'min_plddt': float(plddt_scores.min()),
                'max_plddt': float(plddt_scores.max()),
                'quality': quality
            }
            
            results.append(result)
            
            # Save PDB
            if save_pdbs:
                pdb_filename = f'antibody_{i:03d}_plddt{mean_plddt:.0f}.pdb'
                with open(pdb_filename, 'w') as f:
                    f.write(pdb_string)
            
            print(f"    pLDDT: {mean_plddt:.1f} - {quality}")
            
            # Clean up temp file
            os.unlink(pdb_file)
            
        except Exception as e:
            print(f"    ❌ Error: {e}")
            results.append({'antibody_id': i, 'error': str(e)})
    
    # Summary
    valid_results = [r for r in results if 'error' not in r]
    if valid_results:
        plddt_values = [r['mean_plddt'] for r in valid_results]
        print(f"\n" + "="*70)
        print("Validation Summary")
        print("="*70)
        print(f"Total antibodies:     {len(antibody_sequences)}")
        print(f"Successful:           {len(valid_results)}")
        print(f"Failed:               {len(antibody_sequences) - len(valid_results)}")
        print(f"\nMean pLDDT:           {np.mean(plddt_values):.2f} ± {np.std(plddt_values):.2f}")
        print(f"Median pLDDT:         {np.median(plddt_values):.2f}")
        print(f"Range:                {np.min(plddt_values):.2f} - {np.max(plddt_values):.2f}")
        print("\nQuality Distribution:")
        print(f"  Excellent (>90):    {sum(1 for p in plddt_values if p > 90)} ({sum(1 for p in plddt_values if p > 90)/len(plddt_values)*100:.1f}%)")
        print(f"  Good (70-90):       {sum(1 for p in plddt_values if 70 < p <= 90)} ({sum(1 for p in plddt_values if 70 < p <= 90)/len(plddt_values)*100:.1f}%)")
        print(f"  Fair (50-70):       {sum(1 for p in plddt_values if 50 < p <= 70)} ({sum(1 for p in plddt_values if 50 < p <= 70)/len(plddt_values)*100:.1f}%)")
        print(f"  Poor (<50):         {sum(1 for p in plddt_values if p <= 50)} ({sum(1 for p in plddt_values if p <= 50)/len(plddt_values)*100:.1f}%)")
        print("="*70)
    
    return results

print("✅ Validation function ready!")
print("\nUsage:")
print('  antibodies = [(heavy1, light1), (heavy2, light2), ...]')
print('  results = validate_antibodies(antibodies)')

## 6. Option A: Validate Custom Antibody Sequences

Replace the sequences below with your own antibodies

In [ ]:
# Replace with your generated antibodies
my_antibodies = [
    (
        "EVQLVESGGGLVQPGGSLRLSCAASGFTISDYAIHWVRQAPGKGLEWVAGITPAGGYTAYADSVKGRFTISADTSKNTAYLQMNSLRAEDTAVYYCARFVFFLPYAMDYWGQGTLVTVSS",
        "DIQMTQSPSSLSASVGDRVTITCRASQDVSTAVAWYQQKPGKAPKLLIYSASFLYSGVPSRFSGSGSGTDFTLTISSSLQPEDFATYCQQSYTTPPTFGQGTKVEIKR"
    ),
    # Add more antibodies here...
]

# Run validation
results = validate_antibodies(my_antibodies, save_pdbs=True)

# Display results as table
import pandas as pd
df = pd.DataFrame([r for r in results if 'error' not in r])
print("\n📊 Detailed Results:")
print(df)

## 7. Option B: Upload Model and Generate Antibodies

### 7a. Upload Checkpoint

In [ ]:
from google.colab import files
import os

print("Upload your model checkpoint file:")
print("File: improved_small_2025_10_31_best.pt (~100MB)")
print("\nClick 'Choose Files' below...\n")

uploaded = files.upload()

checkpoint_name = list(uploaded.keys())[0]
print(f"\n✅ Uploaded: {checkpoint_name}")
print(f"   Size: {os.path.getsize(checkpoint_name) / 1e6:.1f} MB")

### 7b. Upload Model Code Files

In [ ]:
from google.colab import files
import shutil

# Create generators directory
!mkdir -p generators

print("Upload your model code files (3 files):")
print("1. transformer_seq2seq.py")
print("2. tokenizer.py")
print("3. data_loader.py")
print("\nClick 'Choose Files' below...\n")

uploaded = files.upload()

# Move files to generators directory
for filename in uploaded.keys():
    shutil.move(filename, f'generators/{filename}')
    print(f"✅ Moved {filename} to generators/")

print("\n✅ All model files uploaded!")

### 7c. Generate Antibodies from Model

In [ ]:
import torch
import sys
sys.path.insert(0, '.')

from generators.transformer_seq2seq import create_model
from generators.tokenizer import AminoAcidTokenizer

print("Loading model and generating antibodies...\n")

# Sample antigens for generation
test_antigens = [
    "MKTIIALSYIFCLVFADYKDDDDK",
    "MKLLILTCLVAVSARAGVEFEGSGA",
    "MNGLKRKTGKVMSKKFTVGLMAMLPT",
    "MKLVVLSLSLVLALILQGVNAFGGGS",
    "MKFLVNVALVFMVVYISYIYAAGSQ",
    "MGAASGRRGPGLLLPLPLLLLLPPQEAL",
    "MKLGIWLALAGLVLAFSQYAFGQG",
    "MLLSVPLLLGLLGLAAADQGVVPR",
    "MKWVTFISLLFLFSSAYSRGVFRRD",
    "MGLSDGEWQLVLNVWGKVEADI"
]

test_pkds = [8.0, 9.0, 7.5, 8.5, 9.5, 8.0, 7.0, 8.5, 9.0, 7.5]  # Target binding affinities

# Load tokenizer
tokenizer = AminoAcidTokenizer()
print(f"✅ Tokenizer loaded (vocab size: {tokenizer.vocab_size})")

# Create and load model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = create_model('small', vocab_size=tokenizer.vocab_size, max_src_len=512, max_tgt_len=300)
model = model.to(device)
print(f"✅ Model created ({model.get_model_size():,} parameters)")

checkpoint = torch.load(checkpoint_name, map_location=device, weights_only=False)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()
print(f"✅ Checkpoint loaded (epoch {checkpoint['epoch']}, val loss {checkpoint['val_loss']:.4f})")

# Generate antibodies
print(f"\n🔬 Generating {len(test_antigens)} antibodies...\n")
generated_antibodies = []

with torch.no_grad():
    for i, (antigen_seq, target_pkd) in enumerate(zip(test_antigens, test_pkds)):
        # Tokenize antigen
        antigen_tokens = tokenizer.encode(antigen_seq)
        if len(antigen_tokens) > 512:
            antigen_tokens = antigen_tokens[:512]
        src = torch.tensor([antigen_tokens]).to(device)
        pkd = torch.tensor([[target_pkd]]).float().to(device)
        
        # Generate
        generated = model.generate_greedy(src, pkd, max_length=300)
        antibody_seq = tokenizer.decode(generated[0].cpu().tolist())
        
        # Split heavy and light chains
        if '|' in antibody_seq:
            heavy, light = antibody_seq.split('|', 1)
        else:
            heavy = antibody_seq[:120]
            light = antibody_seq[120:]
        
        generated_antibodies.append((heavy, light))
        
        if (i + 1) % 5 == 0:
            print(f"   Generated {i + 1}/{len(test_antigens)}")

print(f"\n✅ Generated {len(generated_antibodies)} antibodies!")
print(f"   Average heavy chain length: {sum(len(h) for h, l in generated_antibodies) / len(generated_antibodies):.1f} aa")
print(f"   Average light chain length: {sum(len(l) for h, l in generated_antibodies) / len(generated_antibodies):.1f} aa")

### 7d. Validate Generated Antibodies

In [ ]:
# Validate the generated antibodies
results = validate_antibodies(generated_antibodies, save_pdbs=True)

# Display results
import pandas as pd
df = pd.DataFrame([r for r in results if 'error' not in r])
print("\n📊 Detailed Results:")
print(df.to_string(index=False))

## 8. Download All PDB Structures

In [ ]:
import zipfile
import os
from google.colab import files

# Create zip file with all PDB structures
pdb_files = [f for f in os.listdir('.') if f.endswith('.pdb')]

if pdb_files:
    with zipfile.ZipFile('antibody_structures.zip', 'w') as zipf:
        for pdb_file in pdb_files:
            zipf.write(pdb_file)
    
    print(f"Created zip file with {len(pdb_files)} PDB structures")
    
    # Download
    files.download('antibody_structures.zip')
    print("\n✅ Download complete!")
else:
    print("No PDB files found")

---

## Summary

**What this notebook does:**
- ✅ Uses Google Colab's FREE GPU (T4 with 16GB VRAM)
- ✅ Installs IgFold with all dependencies
- ✅ Fixed pLDDT extraction (proper 0-100 scores)
- ✅ Validates antibody structures
- ✅ Generates antibodies from your trained model
- ✅ Saves PDB structure files
- ✅ Downloads results to your computer

**Usage:**
1. Run Cell 1 → Check GPU
2. Run Cell 2 → Install (then restart!)
3. Run Cell 1 again → Verify GPU
4. Run Cell 3 → Test IgFold
5. Choose:
   - **Option A (Cell 6)**: Validate custom sequences
   - **Option B (Cells 7a-7d)**: Upload model and generate
6. Run Cell 8 → Download all PDB files

**View structures:**
- Online: https://www.rcsb.org/3d-view
- PyMOL: `pymol antibody_000.pdb`
- ChimeraX: https://www.cgl.ucsf.edu/chimerax/
